In [2]:
import os
import pickle
import numpy as np
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

# Paths
data_dir = r"C:\Users\sagni\Downloads\archive (1)\archive\animals\animals\Human_with_gun"
save_dir = r"C:\Users\sagni\Downloads\Aminal Poching Detection"

os.makedirs(save_dir, exist_ok=True)

# Parameters
img_size = (224, 224)
batch_size = 16

# Load dataset
images = []
labels = []

for img_name in os.listdir(data_dir):
    img_path = os.path.join(data_dir, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue
    img = cv2.resize(img, img_size)
    img = img / 255.0
    images.append(img)
    labels.append("Human_with_gun")  # All images are Human_with_gun

images = np.array(images)
labels = np.array(labels)

# Encode labels (binary: poaching or not)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Save label encoder
label_encoder_path = os.path.join(save_dir, "label_encoder.pkl")
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)
print(f"Label encoder saved at: {label_encoder_path}")

# Model
base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))
x = base_model.output
x = Dropout(0.3)(x)
predictions = Dense(1, activation="sigmoid")(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train
model.fit(images, labels_encoded, epochs=5, batch_size=batch_size, verbose=1)

# Save model
model_path = os.path.join(save_dir, "poaching_detection_model.h5")
model.save(model_path)
print(f"Model saved at: {model_path}")


Label encoder saved at: C:\Users\sagni\Downloads\Aminal Poching Detection\label_encoder.pkl
Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 108ms/step - accuracy: 0.9824 - loss: 0.1626
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 1.0000 - loss: 0.0306
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 1.0000 - loss: 0.0054
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 1.0000 - loss: 0.0019
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 1.0000 - loss: 9.5789e-04


Model saved at: C:\Users\sagni\Downloads\Aminal Poching Detection\poaching_detection_model.h5
